In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from find_peak_slope import *
from simulate import *
from zacLib import awgn,mySmooth,IIRsmooth
import numba
%matplotlib Qt5

In [2]:
file = 'siga.rd'
sig,fs = rd_reader(file,1,0)
sp = fft(sig,nextpow2(sig.size,0))
sm = IIRsmooth(sp,5,0.02)
sm.tofile('test.dat')
# sm = mySmooth(sp,149,2)
# sig,sp = stairs()
# sm = np.fromfile('test.dat')

In [10]:
@numba.njit()
def findBandInRange(x,peakThreshold,si,se,minBandPts=3):    
    s = x[si:se]
    mxi = s.argmax()
    i = mxi - 1
    th = s[mxi]-peakThreshold
    while i>=0 and s[i] >= th:
        i -= 1
    j = mxi + 1
    while j<s.size and s[j]>=th:
        j += 1
    peak = [0 for x in range(0)]
    if i<0 and j==s.size:
        return peak
    ipeak = [0 for x in range(0)]
    if i>=minBandPts:
        ipeak = findBandInRange(x,peakThreshold,si,si+i,minBandPts)
    if i>=0 and j<s.size:
        peak = [si+i,si+j]
    jpeak = [0 for x in range(0)]
    if j<s.size-minBandPts:
        jpeak = findBandInRange(x,peakThreshold,si+j,se,minBandPts)    
    ipeak += peak
    ipeak += jpeak
    return ipeak

def foo(sp):
    peaks = findBandInRange(sm,0.5,0,sm.size)
    ploc = peaks[::2]
    vloc = peaks[1::2]
    return ploc,vloc
ploc,vloc = foo(sp)
%timeit findBandInRange(sm,0.5,0,sm.size)

3.74 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
plt.figure()
plt.plot(sp,'.',markersize=0.4)
plt.plot(sm)
plt.plot(ploc,sm[ploc],'r^')
plt.plot(vloc,sm[vloc],'gv')
plt.show()

In [12]:
len(ploc)

24

In [5]:
# @numba.njit()
# def bar():
#     stack = [[0,sm.size]]
#     ploc,vloc = [],[]
#     peakThreshold = 0.5
#     while stack: 
#         start,end = stack.pop()
#         idx = start + sm[start:end].argmax()
#         i,j = idx-1,idx+1
#         th = sm[idx] - peakThreshold
#         while i>=start and sm[i]>=th:
#             i-=1
#         while j<end and sm[j]>=th:
#             j+=1
#         if i>=start and j<end:
#             ploc.append(i)
#             vloc.append(j)
#         if i>=start+3:
#             stack.append([start,i])
#         if j<end-3:
#             stack.append([j,end])
#     ploc.sort()
#     vloc.sort()
#     return ploc,vloc
# ploc1,vloc1 = bar()
# %timeit bar()

In [6]:
# def demo(x,sp,width,slope_th):
    # (sm,dsp,new_peak,new_pits,avg_frq,max_frq,
    #  dev_band,half_left,half_right,half_band) = find_peak_slope(sp,width,slope_th)
     
    # plot_all(sp,sm,dsp,slope_th,new_peak,new_pits,avg_frq,
    #          max_frq,dev_band,half_left,half_right,1,origin=x)

In [7]:
# @numba.njit()
# def cost(u,f,alpha=1):
#     aRu = alpha*np.abs(u[2:]-u[:-2]).sum()
#     DF_Au_f = (np.abs(u.cumsum()-f)**2).sum()*0.5
#     return aRu + DF_Au_f

# @numba.njit()
# def gradient(u,f,alpha):    
#     # d(|u'|)/du
#     sdu = np.sign(u[2:]-u[:-2])
#     aRu = np.zeros_like(u)
#     aRu[2:-2] = sdu[:-2]-sdu[2:]
#     aRu[:2] = -sdu[:2]
#     aRu[-2:] = sdu[-2:]
#     # d(0.5*|Au-f|^2)/du, i.e. At(Au-f)
#     Au_f = u.cumsum() - f
#     At_Au_f = Au_f.cumsum()
#     sumAu_f = At_Au_f[-1]
#     At_Au_f[1:] = At_Au_f[-1]-At_Au_f[:-1]
#     At_Au_f[0] = sumAu_f
#     return alpha*aRu + At_Au_f

# f = sp
# n = sp.size
# alpha = 1
# eta = 4/n**2
# for i in range(50):
#     costs.append(cost(u,f,alpha))
#     grad = gradient(u,f,alpha)
#     u = u - eta * grad

In [8]:
# real data

# find_peak_slope_from_rd('sig0.rd',150,20,1,0)
# find_peak_slope_from_rd('sig1.rd',300,30,1,0)
# find_peak_slope_from_rd('sig2.rd',40,10,1,0)
# find_peak_slope_from_rd('sig3.rd',300,30,1,0)
# find_peak_slope_from_rd('siga.rd',300,30,1,0)

In [9]:
# simulate data

# bell shape
# x,sp = bells(noise=0.3)
# demo(x,sp,200,20)

# square
# x,sp = squares(0.5)
# demo(x,sp,100,20)

#stair
# x,sp = stairs(0.5)
# demo(x,sp,200,30)

# random combination
# x,sp = random(noise=0.7)
# demo(x,sp,100,20)